# Create Course

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [25]:
objectiveTemplate = """
Convert the provided learning objective into a concise, standalone objective, ensuring it is clear, specific, and focused solely on the core intent of the original objective. Learning objective provided: "{objective}" Standalone objective:
"""

courseNameTemplate = """The objective is to create a catchy name for a course from the following learning objective:
    "{learningObjective}".
Write in the same language as the learning objective. You should respond directly with the name and not with a message that contains the name.
"""

courseDescriptionTemplate = """
The objective is to create a short description for a course from the following learning objective:
    "{learningObjective}".

    Write in the same language as the learning objective. The short description should be less than 100 characters. You should respond directly with the short description and not with a message that contains the short description.`
"""

In [30]:

objectivePrompt = ChatPromptTemplate.from_template(objectiveTemplate)
courseNamePrompt = ChatPromptTemplate.from_template(courseNameTemplate)
courseDescriptionPrompt = ChatPromptTemplate.from_template(courseDescriptionTemplate)

model = ChatOpenAI()
output_parser = StrOutputParser()
name_and_desc_chain = RunnableParallel(
    {
        "learningObjective": RunnablePassthrough() | RunnableLambda(lambda x: x["learningObjective"]),
        "name": courseNamePrompt | model | output_parser, 
        "description": courseDescriptionPrompt | model | output_parser, 
    }
)



chain = {"learningObjective": objectivePrompt | model | output_parser} | name_and_desc_chain

objective = chain.invoke({"objective": "bears"})
import json
print(json.dumps(objective, indent=4))

learningObjective = objective["learningObjective"]

{
    "learningObjective": "\"Identify and describe the characteristics and behaviors of bears.\"",
    "name": "\"Bearology: Exploring the Traits and Habits of Bears\"",
    "description": "Identify and describe the characteristics and behaviors of bears."
}


'"Identify and describe the characteristics and behaviors of bears."'

## Create course outline

In [31]:
prompt = ChatPromptTemplate.from_template(
    """
The objective is to create a course outline for a course from the following learning objective:
"{learningObjective}".

Write in the same language as the learning objective. The course outline should be less than 600 characters. You should respond directly with the course outline and not with a message that contains the course outline.
"""
)
model = ChatOpenAI()
output_parser = StrOutputParser()
chain = prompt | model | output_parser

courseOutline = chain.invoke({"learningObjective": learningObjective})

In [32]:
print(courseOutline)

Course Outline:
1. Introduction to Bears
   - Definition and classification of bears
   - Overview of bear species and their distribution
2. Bear Anatomy and Physical Characteristics
   - External features and adaptations
   - Skeletal and muscular system of bears
3. Bear Behavior and Communication
   - Social structure and hierarchy
   - Vocalizations, body language, and scent marking
4. Bear Diet and Feeding Habits
   - Types of food bears consume
   - Hunting, foraging, and scavenging behaviors
5. Bear Reproduction and Life Cycle
   - Mating rituals and breeding patterns
   - Pregnancy, birth, and cub rearing
6. Bear Conservation and Human Interactions
   - Threats to bear populations
   - Conservation efforts and management strategies
7. Bear Safety and Bear-Human Conflict
   - Understanding bear behavior to prevent conflicts
   - Techniques for safe bear encounters and bear spray use
8. Bear Research and Study Opportunities
   - Careers and research in bear biology
   - Fieldwork 

## Create course module text activity

